In [2]:
import numpy as np
import cv2
import pandas as pd

In [87]:
video_name = 'RollerCoaster'
data = pd.read_fwf('data/Subject_1/' + video_name + '.txt', header=None)

data.columns = ['latitude', 'longitude']

cap = cv2.VideoCapture('video_files/' + video_name + '.mp4')

length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

ret, frame = cap.read()

calc_mod = length/len(data)

In [88]:
calc_mod = round(1/(round(calc_mod,2)-1))

In [89]:
frame.shape

(2048, 3640, 3)

In [90]:
def conversion(lat, lon, width, height):
    x = ((lon * -1) + 180) * (width/360)
    y = ((lat) + 90) * (height/180)
    return(x, y)

cap = cv2.VideoCapture('video_files/' + video_name + '.mp4')
index = 0
skip = True
zones = [i for i in range(1,61)]
while(cap.isOpened()):
    ret, frame = cap.read()
    if (((index % calc_mod) == 0) and skip):
        skip = not skip
        index = index - 1

    # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    x, y = conversion(data.iloc[index].latitude, data.iloc[index].longitude, frame.shape[1],frame.shape[0])

    cv2.circle(frame, (np.float32(x), np.float32(y)), 170, (0, 255, 0), 10)
    for i in zones:
        cv2.line(frame, (int((frame.shape[1]/60) * i), 0), (int((frame.shape[1]/60) * i), frame.shape[0]), (255,255,255), 2)
        cv2.line(frame, (0, int(frame.shape[0]/20) * i), (frame.shape[1], int(frame.shape[0]/20) * i), (255,255,255), 2)

    cv2.namedWindow('frame', 0)
    cv2.resizeWindow('frame', 1920,1080)
    cv2.imshow('frame',frame)

    index += 1
    if((index % calc_mod) != 0):
        skip = True
    if (cv2.waitKey(1) & 0xFF == ord('q') or index > len(data)-1):
        break

cap.release()
cv2.destroyAllWindows()

In [23]:
import utm

In [42]:
lst = []
for i in range(len(data)):
    u = utm.from_latlon(data.iloc[i].latitude, data.iloc[i].longitude)
    lst.append(u[2:])

In [28]:
# data_zones = pd.concat([data, pd.DataFrame(lst)],axis=1)
def conversion(lat, lon, width, height):
    x = ((lon * -1) + 180) * (width/360)
    y = ((lat) + 90) * (height/180)
    return(x, y)

scufed = []
for i in range(len(data)):
    scufed.append(conversion(data.iloc[i].latitude, data.iloc[i].longitude, 3840, 2160))

In [44]:
scuffed_xy = pd.DataFrame(lst)

In [47]:
pd.concat([data, scuffed_xy],axis=1)

,latitude,longitude,0,1
0,13.474903,18.634676,34,P
1,13.569384,18.680785,34,P
2,13.672114,18.687172,34,P
3,13.748986,18.684667,34,P
4,13.814718,18.690203,34,P
...,...,...,...,...
613,-0.426411,-67.713565,19,M
614,-0.725175,-67.861549,19,M
615,-0.856629,-67.687099,19,M
616,-0.879255,-67.239816,19,M
